## Task 1 - Bigram Model

Select training data to build a bigram language model that can help you in text sequence generation for sequence data. You should implement a python class with multiple methods to do the following jobs:

1. The class name is __LanguageModel__.
2. A __constructor__ to get the text file name and open that file.
3. A method called __Clean__ to process the text (Tokenization, Lemmatization)! __Don’t remove stop words__.
4. A method called __LMBigram__ to build 2-D Language Model Matrix with Laplace smoothing using NLTK. This method saves generated matrix in an instance variable.
5. A method called __Run__ that takes a part of sentence from user and returns the expected next word using the constructed matrix from the previous step.


__The notebook is composed of three sections__:
1. __Testing code before Class implementation.__
2. __Class Implementation.__
3. __Testing Class Implementation.__


__Assumptions__:
* __URLs__ would be removed
* __Hashtags__ would be removed
* __Mentions__ would be removed
* __Repeating dots__ (more than two occurences) e.g.: __...__ would be removed
* __Emojis__ would be removed
* __Repeating characters__ (more than two occurences) e.g.: عاااااااجل would be removed
* __Arabic Diacritics__ would be removed

In [1]:
import pandas as pd
import glob
import emoji  # for removing emojis
import pyarabic.araby as araby  # for removing diacritics
import re
import qalsadi.lemmatizer
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.arlstem import ARLSTem
from nltk.util import ngrams
from tqdm import tqdm  # progress bar

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/tamer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 1- Code testing before Class implementation

### 1.1- Collecting the data into one dataframe

In [2]:
text_files = glob.glob("data/file*.txt")

In [3]:
text_files

['data/file4.txt',
 'data/file1.txt',
 'data/file10.txt',
 'data/file3.txt',
 'data/file5.txt',
 'data/file2.txt',
 'data/file6.txt',
 'data/file8.txt',
 'data/file7.txt',
 'data/file9.txt']

In [4]:
text_file_df_list = []

for text_file in text_files:
    df = pd.read_csv(text_file, sep="\t")
    text_file_df_list.append(df)
    
full_df = pd.concat(text_file_df_list, ignore_index=True)    

In [5]:
full_df

,tweetID,tweetText
0,553215687247015936,حال استنفار قصوى في شمال فرنسا http://t.co/umT...
1,553215784131252224,عَمْرُو عبد الهادي الاعلام الصهيوني المصري مش ...
2,553215847792009216,عرض فرصه للبيع مزرعه من الوكيل مباشر في الدائر...
3,553215853580529664,10 قتلى بانفجارين أحدهما بمسجد شيعي في بغداد h...
4,553216005913452545,@teriangle_hd القنابل البدائية مش من مقام فرنس...
...,...,...
49995,560873525779496960,السيادة على مزارع شبعا http://t.co/LaUZcJYZii ...
49996,560873585296687104,"#عاجل أوامر ملكية بعد قليل على السعودية الأولى"""
49997,560873611091644416,عاجل/ أوامر ملكية بعد قليل على قناة السعودية ا...
49998,560873618435891200,#الرياض الميليشيات تبايع نوري المالكي!: الميلي...


In [6]:
full_df.to_csv("data/full_data.txt", index=None, sep="\t")

In [7]:
# testing reading the created file containing all tweets
full_df = pd.read_csv("data/full_data.txt", sep="\t")

In [8]:
full_df

,tweetID,tweetText
0,553215687247015936,حال استنفار قصوى في شمال فرنسا http://t.co/umT...
1,553215784131252224,عَمْرُو عبد الهادي الاعلام الصهيوني المصري مش ...
2,553215847792009216,عرض فرصه للبيع مزرعه من الوكيل مباشر في الدائر...
3,553215853580529664,10 قتلى بانفجارين أحدهما بمسجد شيعي في بغداد h...
4,553216005913452545,@teriangle_hd القنابل البدائية مش من مقام فرنس...
...,...,...
49995,560873525779496960,السيادة على مزارع شبعا http://t.co/LaUZcJYZii ...
49996,560873585296687104,"#عاجل أوامر ملكية بعد قليل على السعودية الأولى"""
49997,560873611091644416,عاجل/ أوامر ملكية بعد قليل على قناة السعودية ا...
49998,560873618435891200,#الرياض الميليشيات تبايع نوري المالكي!: الميلي...


### 1.2- Extracting text from full_df

In [9]:
full_df.drop("tweetID", axis=1, inplace=True)

In [10]:
full_df

,tweetText
0,حال استنفار قصوى في شمال فرنسا http://t.co/umT...
1,عَمْرُو عبد الهادي الاعلام الصهيوني المصري مش ...
2,عرض فرصه للبيع مزرعه من الوكيل مباشر في الدائر...
3,10 قتلى بانفجارين أحدهما بمسجد شيعي في بغداد h...
4,@teriangle_hd القنابل البدائية مش من مقام فرنس...
...,...
49995,السيادة على مزارع شبعا http://t.co/LaUZcJYZii ...
49996,"#عاجل أوامر ملكية بعد قليل على السعودية الأولى"""
49997,عاجل/ أوامر ملكية بعد قليل على قناة السعودية ا...
49998,#الرياض الميليشيات تبايع نوري المالكي!: الميلي...


In [11]:
test_sent = full_df.iloc[0][0]

In [12]:
test_sent

'حال استنفار قصوى في شمال فرنسا http://t.co/umTkhJyKkn'

### 1.3- Text Pre-processing

Assumptions:
* __URLs__ would be removed
* __Hashtags__ would be removed
* __Mentions__ would be removed
* __Repeating dots__ (more than two occurences) e.g.: __...__ would be removed
* __Emojis__ would be removed
* __Repeating characters__ (more than two occurences) e.g.: عاااااااجل would be removed
* __Arabic Diacritics__ would be removed

##### 1.3.1 Testing regular expression substitution

In [13]:
# Removing URLs, Hashtags, Mentions, and Repeating dots
#  removing URLs from text StackOverflow reference: https://stackoverflow.com/a/11332580
#  removing repeating characters than occur more than two times StackOverflow reference: https://stackoverflow.com/a/4574516
#   URL: https?:\/\/.*[\r\n]*
#   Hashtag: #\w+
#   Mention: @\w+
#   Repeating dots: \.{2,}
#   Repeating character: (\w)\1{2,}

# Note: the pattern needs to be in raw string format
re_general_pattern = r"https?:\/\/.*[\r\n]*|#\w+|@\w+|\.{2,}"
re_repeating_character_pattern = r"(\w)\1{2,}"

In [14]:
# Removing arabic diacritics
#  StackOverflow reference: https://stackoverflow.com/a/66988480/10086080
def remove_diacritics(string):
    return araby.strip_diacritics(string)

In [15]:
# Removing emojis
#  StackOverflow reference: https://stackoverflow.com/a/67396231
def remove_emojis(string):
    return emoji.get_emoji_regexp().sub(u"", string)

In [16]:
emoji.get_emoji_regexp()

/home/tamer/anaconda3/envs/psut-nlp-course/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  """Entry point for launching an IPython kernel.


re.compile(r'(👨🏿\u200d❤️\u200d💋\u200d👨🏿|👨🏿\u200d❤️\u200d💋\u200d👨🏻|👨🏿\u200d❤️\u200d💋\u200d👨🏾|👨🏿\u200d❤️\u200d💋\u200d👨🏼|👨🏿\u200d❤️\u200d💋\u200d👨🏽|👨🏻\u200d❤️\u200d💋\u200d👨🏻|👨🏻\u200d❤️\u200d💋\u200d👨🏿|👨🏻\u200d❤️\u200d💋\u200d👨🏾|👨🏻\u200d❤️\u200d💋\u200d👨🏼|👨🏻\u200d❤️\u200d💋\u200d👨🏽|👨🏾\u200d❤️\u200d💋\u200d👨🏾|👨🏾\u200d❤️\u200d💋\u200d👨🏿|👨🏾\u200d❤️\u200d💋\u200d👨🏻|👨🏾\u200d❤️\u200d💋\u200d👨🏼|👨🏾\u200d❤️\u200d💋\u200d👨🏽|👨🏼\u200d❤️\u200d💋\u200d👨🏼|👨🏼\u200d❤️\u200d💋\u200d👨🏿|👨🏼\u200d❤️\u200d💋\u200d👨🏻|👨🏼\u200d❤️\u200d💋\u200d👨🏾|👨🏼\u200d❤️\u200d💋\u200d👨🏽|👨🏽\u200d❤️\u200d💋\u200d👨🏽|👨🏽\u200d❤️\u200d💋\u200d👨🏿|👨🏽\u200d❤️\u200d💋\u200d👨🏻|👨🏽\u200d❤️\u200d💋\u200d👨🏾|👨🏽\u200d❤️\u200d💋\u200d👨🏼|🧑🏿\u200d❤️\u200d💋\u200d🧑🏻|🧑🏿\u200d❤️\u200d💋\u200d🧑🏾|🧑🏿\u200d❤️\u200d💋\u200d🧑🏼|🧑🏿\u200d❤️\u200d💋\u200d🧑🏽|🧑🏻\u200d❤️\u200d💋\u200d🧑🏿|🧑🏻\u200d❤️\u200d💋\u200d🧑🏾|🧑🏻\u200d❤️\u200d💋\u200d🧑🏼|🧑🏻\u200d❤️\u200d💋\u200d🧑🏽|🧑🏾\u200d❤️\u200d💋\u200d🧑🏿|🧑🏾\u200d❤️\u200d💋\u200d🧑🏻|🧑🏾\u200d❤️\u200d💋\u200d🧑🏼|🧑🏾\u200d❤️\u200d💋\u200d🧑🏽|🧑🏼\u200d❤️\u200d💋\u200d🧑

###### 1.3.1.1 Testing removing repeating characters

In [17]:
re.sub(re_repeating_character_pattern, r"\1", "عااااااجل خطييير ناااااار moooooo nice yumyyyy nee")

'عاجل خطير نار mo nice yumy nee'

###### 1.3.1.2 Testing removing Emojis

In [18]:
full_df.iloc[49974][0]

': #عاجل ⭕️ #الملك_سلمان يصدر قرارات ملكية يتم الان اعلانها على القناة #السعودية"'

In [19]:
test_sent = remove_emojis(full_df.iloc[49974][0])
test_sent

/home/tamer/anaconda3/envs/psut-nlp-course/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  after removing the cwd from sys.path.


': #عاجل ️ #الملك_سلمان يصدر قرارات ملكية يتم الان اعلانها على القناة #السعودية"'

In [20]:
test_sent = re.sub(re_general_pattern, "", test_sent)
test_sent = re.sub(re_repeating_character_pattern, r"\1", test_sent)
test_sent

':  ️  يصدر قرارات ملكية يتم الان اعلانها على القناة "'

###### 1.3.1.3 Testing removing Mentions

In [21]:
# Removing mentions from text
full_df.iloc[49994][0]

'“@AlArabiya_Brk: السعودية: الملك سلمان يصدر قرارات ملكية تعلن بعد قليل”'

In [22]:
test_sent2 = remove_emojis(full_df.iloc[49994][0])
test_sent2 = re.sub(re_general_pattern, "", test_sent2)
test_sent2 = re.sub(re_repeating_character_pattern, r"\1", test_sent2)

/home/tamer/anaconda3/envs/psut-nlp-course/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  after removing the cwd from sys.path.


In [23]:
test_sent2

'“: السعودية: الملك سلمان يصدر قرارات ملكية تعلن بعد قليل”'

###### 1.3.1.4 Testing removing URLs

In [24]:
full_df.iloc[0][0]

'حال استنفار قصوى في شمال فرنسا http://t.co/umTkhJyKkn'

In [25]:
test_sent3 = remove_emojis(full_df.iloc[0][0])
test_sent3 = re.sub(re_general_pattern, "", test_sent3)
test_sent3 = re.sub(re_repeating_character_pattern, r"\1", test_sent3)

/home/tamer/anaconda3/envs/psut-nlp-course/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  after removing the cwd from sys.path.


In [26]:
test_sent3

'حال استنفار قصوى في شمال فرنسا '

###### 1.3.1.5 Testing removing Hashtags

In [27]:
# Removing Hashtags from text
full_df.iloc[49971][0]

'#Qatar الميليشيات تبايع نوري المالكي!: الميليشيات تبايع نوري المالكي!التاريخ: 2015-01-29 في خط... http://t.co/5ibH7IezDp #Oman'

In [28]:
test_sent4 = remove_emojis(full_df.iloc[49971][0])
test_sent4 = re.sub(re_general_pattern, "", test_sent4)
test_sent4 = re.sub(re_repeating_character_pattern, r"\1", test_sent4)
test_sent4

/home/tamer/anaconda3/envs/psut-nlp-course/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  after removing the cwd from sys.path.


' الميليشيات تبايع نوري المالكي!: الميليشيات تبايع نوري المالكي!التاريخ: 2015-01-29 في خط '

###### 1.3.1.6 Testing removing Diacritics

In [29]:
remove_diacritics("اللَّهمَّ اغْفِرْ لنَا ولوالدِينَا")

'اللهم اغفر لنا ولوالدينا'

### 1.4 Testing Word Tokenization

Assumptions:
* __Punctuations__ would be removed
    * Punctuation removal StackOverflow Reference: https://stackoverflow.com/a/41024113

In [30]:
test_sent

':  ️  يصدر قرارات ملكية يتم الان اعلانها على القناة "'

In [31]:
word_tokenize(test_sent)

[':',
 '️',
 'يصدر',
 'قرارات',
 'ملكية',
 'يتم',
 'الان',
 'اعلانها',
 'على',
 'القناة',
 '``']

In [32]:
# Punctuation removal
[word.lower() for word in word_tokenize(test_sent) if word.isalpha()]

['يصدر', 'قرارات', 'ملكية', 'يتم', 'الان', 'اعلانها', 'على', 'القناة']

#### 1.4.1 Testing punctuation removal

In [33]:
test_sent

':  ️  يصدر قرارات ملكية يتم الان اعلانها على القناة "'

In [34]:
test_sent = [word.lower() for word in word_tokenize(test_sent) if word.isalpha()]
test_sent = " ".join(test_sent)
test_sent

'يصدر قرارات ملكية يتم الان اعلانها على القناة'

In [35]:
test_sent2

'“: السعودية: الملك سلمان يصدر قرارات ملكية تعلن بعد قليل”'

In [36]:
test_sent2 = [word.lower() for word in word_tokenize(test_sent2) if word.isalpha()]
test_sent2 = " ".join(test_sent2)
test_sent2

'السعودية الملك سلمان يصدر قرارات ملكية تعلن بعد قليل'

In [37]:
test_sent3

'حال استنفار قصوى في شمال فرنسا '

In [38]:
test_sent3 = [word.lower() for word in word_tokenize(test_sent3) if word.isalpha()]
test_sent3 = " ".join(test_sent3)
test_sent3

'حال استنفار قصوى في شمال فرنسا'

In [39]:
test_sent4

' الميليشيات تبايع نوري المالكي!: الميليشيات تبايع نوري المالكي!التاريخ: 2015-01-29 في خط '

In [40]:
test_sent4 = [word.lower() for word in word_tokenize(test_sent4) if word.isalpha()]
test_sent4 = " ".join(test_sent4)
test_sent4

'الميليشيات تبايع نوري المالكي الميليشيات تبايع نوري المالكي التاريخ في خط'

### 1.5 Testing Stemming/Lemmatizing Arabic Language

#### 1.5.1 Testing ARLSTem for stemming Arabic Language

In [41]:
ARstemmer = ARLSTem()

In [42]:
test_sent

'يصدر قرارات ملكية يتم الان اعلانها على القناة'

In [43]:
singles = [ARstemmer.stem(token) for token in word_tokenize(test_sent)]
print(" ".join(singles))

صدر قرار ملكي يتم الن اعل علي قنا


In [44]:
test_sent2

'السعودية الملك سلمان يصدر قرارات ملكية تعلن بعد قليل'

In [45]:
singles = [ARstemmer.stem(token) for token in word_tokenize(test_sent2)]
print(" ".join(singles))

سعودي ملك سلم صدر قرار ملكي تعل بعد قليل


In [46]:
test_sent3

'حال استنفار قصوى في شمال فرنسا'

In [47]:
singles = [ARstemmer.stem(token) for token in word_tokenize(test_sent3)]
print(" ".join(singles))

حال ستنفر قصوي في شمال فرنس


In [48]:
test_sent4

'الميليشيات تبايع نوري المالكي الميليشيات تبايع نوري المالكي التاريخ في خط'

In [49]:
singles = [ARstemmer.stem(token) for token in word_tokenize(test_sent4)]
print(" ".join(singles))

ميليشي بايع وري مال ميليشي بايع وري مال تاريخ في خط


#### 1.5.2 Testing Qalsadi Lemmatizer for lemmatizing Arabic Language

In [50]:
qalsadi_lemmatizer = qalsadi.lemmatizer.Lemmatizer()

In [51]:
test_sent

'يصدر قرارات ملكية يتم الان اعلانها على القناة'

In [52]:
singles = [qalsadi_lemmatizer.lemmatize(token) for token in word_tokenize(test_sent)]
print(" ".join(singles))

صدر قرار ملك يتم ال اعلانها على قناة


In [53]:
test_sent2

'السعودية الملك سلمان يصدر قرارات ملكية تعلن بعد قليل'

In [54]:
singles = [qalsadi_lemmatizer.lemmatize(token) for token in word_tokenize(test_sent2)]
print(" ".join(singles))

سعودي ملك سلم صدر قرار ملك علن بعد قليل


In [55]:
test_sent3

'حال استنفار قصوى في شمال فرنسا'

In [56]:
singles = [qalsadi_lemmatizer.lemmatize(token) for token in word_tokenize(test_sent3)]
print(" ".join(singles))

حال استنفار قصوى في شمال فرنسا


In [57]:
test_sent4

'الميليشيات تبايع نوري المالكي الميليشيات تبايع نوري المالكي التاريخ في خط'

In [58]:
singles = [qalsadi_lemmatizer.lemmatize(token) for token in word_tokenize(test_sent4)]
print(" ".join(singles))

الميليشيات بايع نور مالك الميليشيات بايع نور مالك تاريخ في خط


__The Qalsadi Lemmatizer will be chosen for the Bigram Language Model__

In [59]:
test_sent = [qalsadi_lemmatizer.lemmatize(token) for token in word_tokenize(test_sent)]
print(" ".join(test_sent))

صدر قرار ملك يتم ال اعلانها على قناة


In [60]:
test_sent2 = [qalsadi_lemmatizer.lemmatize(token) for token in word_tokenize(test_sent2)]
print(" ".join(test_sent2))

سعودي ملك سلم صدر قرار ملك علن بعد قليل


In [61]:
test_sent3 = [qalsadi_lemmatizer.lemmatize(token) for token in word_tokenize(test_sent3)]
print(" ".join(test_sent3))

حال استنفار قصوى في شمال فرنسا


In [62]:
test_sent4 = [qalsadi_lemmatizer.lemmatize(token) for token in word_tokenize(test_sent4)]
print(" ".join(test_sent4))

الميليشيات بايع نور مالك الميليشيات بايع نور مالك تاريخ في خط


### 1.6 Testing bigram model creation

#### 1.6.1 Testing nltk bigram function

In [63]:
test_sent

['صدر', 'قرار', 'ملك', 'يتم', 'ال', 'اعلانها', 'على', 'قناة']

In [64]:
list(ngrams(test_sent, 2))

[('صدر', 'قرار'),
 ('قرار', 'ملك'),
 ('ملك', 'يتم'),
 ('يتم', 'ال'),
 ('ال', 'اعلانها'),
 ('اعلانها', 'على'),
 ('على', 'قناة')]

In [65]:
test_list = []

In [66]:
test_list.extend(list(ngrams(test_sent, 2)))

In [67]:
test_list

[('صدر', 'قرار'),
 ('قرار', 'ملك'),
 ('ملك', 'يتم'),
 ('يتم', 'ال'),
 ('ال', 'اعلانها'),
 ('اعلانها', 'على'),
 ('على', 'قناة')]

In [68]:
test_list.extend(list(ngrams(test_sent2, 2)))

In [69]:
test_list

[('صدر', 'قرار'),
 ('قرار', 'ملك'),
 ('ملك', 'يتم'),
 ('يتم', 'ال'),
 ('ال', 'اعلانها'),
 ('اعلانها', 'على'),
 ('على', 'قناة'),
 ('سعودي', 'ملك'),
 ('ملك', 'سلم'),
 ('سلم', 'صدر'),
 ('صدر', 'قرار'),
 ('قرار', 'ملك'),
 ('ملك', 'علن'),
 ('علن', 'بعد'),
 ('بعد', 'قليل')]

#### 1.6.2 Testing the text processing function clean() - Steps 1.1 to 1.5

In [70]:
# Creating text preprocessing function (clean()) from steps 1.1 to 1.5
def clean(sentence):
    def remove_emojis(string):
        return emoji.get_emoji_regexp().sub(u"", string)
    
    def remove_diacritics(string):
        return araby.strip_diacritics(string)

    re_general_pattern = r"https?:\/\/.*[\r\n]*|#\w+|@\w+|\.{2,}"
    re_repeating_character_pattern = r"(\w)\1{2,}"
    lemmatizer = qalsadi.lemmatizer.Lemmatizer()
    
    # 1- Removing URLs, Hashtags, Mentions, and repeating dots
    sentence = re.sub(re_general_pattern, "", sentence)
    # 2- Removing repeating characters that occur more than twice
    sentence = re.sub(re_repeating_character_pattern, r"\1", sentence)
    # 3- Removing emojis
    sentence = remove_emojis(sentence)
    # 4- Removing arabic diacritics
    sentence = remove_diacritics(sentence)
    # 5- Tokenization and punctuation removal (only alphanumeric)
    sequence = [token.lower() for token in word_tokenize(sentence) if token.isalpha()]
    # 6- Lemmatization
    sequence = [qalsadi_lemmatizer.lemmatize(token) for token in sequence]
    
    return sequence

In [71]:
full_df.iloc[0][0]

'حال استنفار قصوى في شمال فرنسا http://t.co/umTkhJyKkn'

In [72]:
clean(full_df.iloc[0][0])

/home/tamer/anaconda3/envs/psut-nlp-course/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  after removing the cwd from sys.path.


['حال', 'استنفار', 'قصوى', 'في', 'شمال', 'فرنسا']

#### 1.6.3 Testing bigram model creation from tweet data

In [73]:
test_df = pd.read_csv("data/file1.txt", sep="\t").drop("tweetID", axis=1)

In [74]:
test_df

,tweetText
0,"الاعدام لعامل مطعم قتل زميله طعناً في ""البيادر..."
1,#الأخبار ▪ تأجيل محاكمة 7 إرهابيين بسبب غياب ا...
2,@helale9999 عشآن أعطيتك وحده صميم صرت ترمي أعذ...
3,#النهدي ثمانية قتلى في تفجير انتحاري بسيارة مف...
4,البحرين: ضبط مطلوبين متورطين في التفجير بالعكر...
...,...
4995,«بوكو حرام »تخطف 40 شابا في شمال شرق نيجيريا -...
4996,قصيدة مهداة إلى الشيخ/ ضيف الله بن سمار كلمات/...
4997,العفو عند المقبرة!!
4998,#للمرة الأولى منذ 92 عاما.. تركيا تسمح ببناء ك...


In [75]:
for index, tweet in test_df.iterrows():
    if index == 10:
        break
    print(tweet[0])

الاعدام لعامل مطعم قتل زميله طعناً في "البيادر" أيدت محكمة التمييز الحكم الصادر عن محكمة الجنايات الكبرى والقاضي... http://t.co/H0txdjv3Kn
#الأخبار ▪ تأجيل محاكمة 7 إرهابيين بسبب غياب الدفاع: أجلت محكمة الجنايات بالعاصمة إلى تاريخ لاحق محاكمة سبعة إ... http://t.co/GM4jmpAWbR
@helale9999 عشآن أعطيتك وحده صميم صرت ترمي أعذار ...حقق العالميةة و أرجع كلمني يَ الأياب الانتحاري
#النهدي ثمانية قتلى في تفجير انتحاري بسيارة مفخخة أمام معملين للغاز في ريف حمص - شبكة الصين http://t.co/r5zFEuzAPu
البحرين: ضبط مطلوبين متورطين في التفجير بالعكر الشرقي بقية الموضوع اضغط هنا http://t.co/t4A5bNrqyh
@El__DoN__ABoOoD @soliman_sport توهير تفكيره يورو على يورو يطلع ٢ يورو وعلى كذا ثقافة آسيوية متأصلة وبصراحة أشوف إنها جيدة له لوحده فقط
صحيفة "نيويورك تايمز" الطفل الانتحاري الذي سلم نفسه لحظات قبل التفجير قرب حسينيه في البياع هو سوري وليس عراقي... http://t.co/PH1a97rvtb
ليتوانيا تستقبل عام 2015 باعتماد اليورو http://t.co/29IJLgr73b
يحتاج الداعي أكثر من غيره إلى حسن الظاهر والوقار والهدوء ليتأسى به ، وخير ال

In [76]:
bigrams = []
unigrams = []  # for laplacian smoothing

for index, tweet in tqdm(test_df.iterrows()):
    sequence = clean(tweet[0]) 
    
    # bigram model creation
    bigrams.extend(list(ngrams(sequence, 2)))
    
    # unigram model creation for the purposes of laplacian smoothing
    unigrams.extend(list(ngrams(sequence, 1)))

0it [00:00, ?it/s]/home/tamer/anaconda3/envs/psut-nlp-course/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  after removing the cwd from sys.path.
5000it [02:32, 32.87it/s]


In [77]:
freq_bi = nltk.FreqDist(bigrams)
freq_uni = nltk.FreqDist(unigrams)

print ("Most common bigrams without stopword removal and without add-1 smoothing: ", freq_bi.most_common(10))
print ("Most common unigrams without stopword removal and without add-1 smoothing: ", freq_uni.most_common(10))

Most common bigrams without stopword removal and without add-1 smoothing:  [(('أنس', 'ليبي'), 420), (('تفجير', 'انتحار'), 418), (('أبو', 'أنس'), 398), (('في', 'تفجير'), 237), (('من', 'محاكم'), 236), (('كور', 'شمال'), 232), (('في', 'نيويورك'), 232), (('على', 'كور'), 225), (('قياد', 'في'), 209), (('جنائي', 'دولي'), 204)]
Most common unigrams without stopword removal and without add-1 smoothing:  [(('في',), 2725), (('على',), 1345), (('من',), 1048), (('تفجير',), 886), (('سوى',), 684), (('ليبي',), 598), (('انتحار',), 580), (('أنس',), 536), (('أبو',), 501), (('كنيس',), 459)]


#### 1.6.4 Testing bigram model with laplacian smoothing (add-1 smoothing)

Note: I am assuming the vocabulary size would be the number of unique n-1 grams according to this quora thread [link](https://www.quora.com/What-is-the-meaning-of-Vocabulary-in-n-gram-Laplace-Smoothing)

In [78]:
# Creating unigram vocabulary set for laplacian smoothing
unigrams_voc = set([])

for unigram in unigrams:
    if unigram not in unigrams_voc:
        unigrams_voc.add(unigram)

In [79]:
vocab_size = len(unigrams_voc)
vocab_size

5934

In [80]:
len(bigrams)

52826

In [81]:
test_bigram = ('أنس', 'ليبي')

In [82]:
bigrams.count(test_bigram)

420

In [83]:
test_bigram[0]

'أنس'

In [84]:
# Needs to be as a tuple instead of a string
(test_bigram[0],)

('أنس',)

In [85]:
unigrams.count((test_bigram[0],))

536

In [86]:
bigrams_smoothed_prob_model = {}

for bigram in tqdm(bigrams):
    numerator = bigrams.count(bigram) + 1  # Count of (wi-1, wi)
    denominator = unigrams.count((bigram[0],)) + vocab_size  # Count of wi - 1
    smoothed_prob = numerator / denominator
    
    bigrams_smoothed_prob_model[bigram] = smoothed_prob

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 52826/52826 [04:43<00:00, 186.12it/s]


In [87]:
bigrams_smoothed_prob_model = dict(sorted(bigrams_smoothed_prob_model.items(), key=lambda item: item[1], reverse=True))

In [88]:
list(bigrams_smoothed_prob_model.items())[:10]

[(('أنس', 'ليبي'), 0.06506955177743431),
 (('أبو', 'أنس'), 0.062004662004662),
 (('تفجير', 'انتحار'), 0.06143695014662757),
 (('كور', 'شمال'), 0.037562469772690635),
 (('من', 'محاكم'), 0.03394442853050702),
 (('قياد', 'في'), 0.0337783496863439),
 (('جنائي', 'دولي'), 0.033311667208319795),
 (('أيام', 'من'), 0.03220559531554977),
 (('على', 'كور'), 0.031048220909465588),
 (('قبل', 'أيام'), 0.03100409191060749)]

In [89]:
bigrams_smoothed_prob_model[('أنس', 'ليبي')]

0.06506955177743431

#### 1.6.5 Testing next word prediction

Assumption: According to my understanding of the implementation of the code, for bigram prediction for the next word; the sorted list of bigrams based on probability would be iterated through from highest to lowest:
* The last bigram of the input sentence would be used for checking with the sorted list of bigrams, the second word of the last bigram would be checked if it was the first word occuring in the list of sorted bigrams to "predict" the most possible next word

In [90]:
test_bigrams_model = list(bigrams_smoothed_prob_model.items())

In [91]:
test_bigrams_model

[(('أنس', 'ليبي'), 0.06506955177743431),
 (('أبو', 'أنس'), 0.062004662004662),
 (('تفجير', 'انتحار'), 0.06143695014662757),
 (('كور', 'شمال'), 0.037562469772690635),
 (('من', 'محاكم'), 0.03394442853050702),
 (('قياد', 'في'), 0.0337783496863439),
 (('جنائي', 'دولي'), 0.033311667208319795),
 (('أيام', 'من'), 0.03220559531554977),
 (('على', 'كور'), 0.031048220909465588),
 (('قبل', 'أيام'), 0.03100409191060749),
 (('ليبي', 'قبل'), 0.03046540110226577),
 (('محكم', 'جنائي'), 0.029695150845048177),
 (('محكم', 'جناية'), 0.027799715684725952),
 (('وفاة', 'أبو'), 0.02774729581439097),
 (('في', 'تفجير'), 0.0274858528698464),
 (('مر', 'أولى'), 0.027294882209585702),
 (('وفاة', 'قياد'), 0.027277002664994512),
 (('في', 'نيويورك'), 0.0269084189860261),
 (('لبن', 'فرض'), 0.026383943023632242),
 (('على', 'سوري'), 0.025965105096853965),
 (('مركز', 'ثقاف'), 0.024737876802096985),
 (('منطق', 'يورو'), 0.02430668841761827),
 (('بوكو', 'حرام'), 0.02401315789473684),
 (('دخول', 'على'), 0.022789720381444965),


In [92]:
test_string = "الاعدام لعامل مطعم قتل"
clean(test_string)

/home/tamer/anaconda3/envs/psut-nlp-course/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  after removing the cwd from sys.path.


['الاعدام', 'عامل', 'مطعم', 'قتل']

In [93]:
test_string_bigrams = list(ngrams(clean(test_string), 2))
test_string_bigrams

/home/tamer/anaconda3/envs/psut-nlp-course/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  after removing the cwd from sys.path.


[('الاعدام', 'عامل'), ('عامل', 'مطعم'), ('مطعم', 'قتل')]

In [94]:
test_string_bigrams[-1]

('مطعم', 'قتل')

In [95]:
test_string_bigrams[-1][0]

'مطعم'

In [96]:
test_string_bigrams[-1][1]

'قتل'

In [97]:
test_bigrams_model[0][0]

('أنس', 'ليبي')

In [98]:
test_bigrams_model[0][0][0]

'أنس'

In [99]:
test_bigrams_model[0][0][1]

'ليبي'

In [100]:
pred = []

count = 0
for bigram in test_bigrams_model:
    if bigram[0][0] == test_string_bigrams[-1][1]:      
        # to find predictions based on highest probability of n-grams     
        count += 1
        pred.append(bigram[0][1])  # Append second word in the bigram as the predicted word
        if count == 5:
            break
            
if count < 5:
    while(count != 5):
        pred.append("\0")           
        # if no word prediction is found, replace with "\0"
        count += 1

In [101]:
pred

['عشر', 'على', 'شخص', 'ملك', 'رب']

In [102]:
pred[0]

'عشر'

## 2- Class code

In [103]:
class LanguageModel:
    def __init__(self, textfile):
        """Creates a bigram model with Laplacian Smoothing based on input textfile containing Arabic tweet
        dataset with two columns: "tweetID" and "tweetText.
        
        Args:
            textfile (string): relative or absolute path to input textfile.
        """
        self.bigrams = []
        self.unigrams = []  # For Laplacian Smoothing
        self.unigrams_voc = set([])  # For calculating unigram vocab size for Laplacian Smoothing
        self.lemmatizer = qalsadi.lemmatizer.Lemmatizer()
        
        # Creating Laplacian Smoothed bigrams probability model
        df = pd.read_csv(textfile, sep="\t").drop("tweetID", axis=1)
        self.bigrams_prob_model = self.lm_bigram(df)
        
    
    def clean(self, sentence):
        """Pre-processes an Arabic sentence for bigram modelling.
        
        1- Removes URLs, Hashtags, Mentions, and repeating dots.
        2- Removes repeating characters than occur more than twice.
        3- Removes emojis.
        4- Removes Arabic Diacritics.
        5- Removes punctuations and only keeps alphanumeric characters.
        6- Converts sentence to Word Tokens.
        7- Lemmatizes each Token using the Qalsadi Lemmatizer.
        
        Args:
            sentence (string): Arabic sentence to preprocess.
            
        Returns:
            sequence (list): processed list of tokens. 
        """
        def _remove_emojis(string):
            return emoji.get_emoji_regexp().sub(u"", string)
        
        def _remove_diacritics(string):
            return araby.strip_diacritics(string)

        re_general_pattern = r"https?:\/\/.*[\r\n]*|#\w+|@\w+|\.{2,}"
        re_repeating_character_pattern = r"(\w)\1{2,}"

        # 1- Removing URLs, Hashtags, Mentions, and repeating dots
        sentence = re.sub(re_general_pattern, "", sentence)
        # 2- Removing repeating characters that occur more than twice
        sentence = re.sub(re_repeating_character_pattern, r"\1", sentence)
        # 3- Removing emojis
        sentence = _remove_emojis(sentence)
        # 4- Removing arabic diacritics
        sentence = _remove_diacritics(sentence)
        # 5- Tokenization and punctuation removal (only alphanumeric)
        sequence = [token.lower() for token in word_tokenize(sentence) if token.isalpha()]
        # 6- Lemmatization
        sequence = [qalsadi_lemmatizer.lemmatize(token) for token in sequence]

        return sequence
    
    def lm_bigram(self, df):
        """Creates Bigram model with Laplacian Smoothing.
        
        Args:
            df (pandas.DataFrame): Dataframe containing Arabic tweet dataset with two columns
            "tweetID" and "tweetText".
        
        Returns:
            bigrams_prob_model (list): sorted list of laplacian smoothed probabilities of the bigrams
                                       in the dataset.
        """
        
        print("Creating bigrams and unigrams list ...")
        for index, tweet in tqdm(df.iterrows()):
            sequence = self.clean(tweet[0]) 
            
            # bigram model creation
            self.bigrams.extend(list(ngrams(sequence, 2)))

            # unigram model creation for the purposes of laplacian smoothing
            self.unigrams.extend(list(ngrams(sequence, 1)))
        
        # Laplacian Smoothing
        print("Laplacian smoothing ...")
        # 1- Creating unigram vocabulary set for laplacian smoothing
        for unigram in self.unigrams:
            if unigram not in self.unigrams_voc:
                self.unigrams_voc.add(unigram)
        
        # 2- Creating laplacian smoothed bigram probability model
        bigrams_smoothed_prob_model = {}

        for bigram in tqdm(self.bigrams):
            numerator = self.bigrams.count(bigram) + 1  # Count of (wi-1, wi)
            denominator = self.unigrams.count((bigram[0],)) + vocab_size  # Count of wi - 1
            smoothed_prob = numerator / denominator

            bigrams_smoothed_prob_model[bigram] = smoothed_prob
        
        # 3- Sorting the smoothed prob model by highest probability
        bigrams_smoothed_prob_model = dict(sorted(bigrams_smoothed_prob_model.items(), key=lambda item: item[1], reverse=True))
        bigrams_prob_model = list(bigrams_smoothed_prob_model.items())
        
        return bigrams_prob_model
        
    
    def run(self, input_sentence, n_words=5):
        """Predicts the most likely next n words in an Arabic input sentence based on the
        created bigram model. Appends "\0" if no word prediction is found.
        
        Args:
            input_sentence (string): input Arabic sentence to predict the next most likely
                                      word for.
            n_words (int): number of top next possible words to return.
        
        Returns:
            pred (list): list of top n_words possible next words for the Arabic input sentence.
        """
        pred = []
        
        input_string_bigrams = list(ngrams(clean(input_sentence), 2))

        count = 0
        for bigram in self.bigrams_prob_model:
            # Checking second word in the input sentence last bigram with the first word in the list of bigrams
            #  sorted on highest probability
            if bigram[0][0] == input_string_bigrams[-1][1]:        
                count += 1
                pred.append(bigram[0][1])  # Append second word in the highest probability bigram as the predicted word
                if count == n_words:
                    break

        if count < n_words:
            while(count != n_words):
                pred.append("\0")           
                # if no word prediction is found, replace with "\0"
                count += 1
        
        return pred

## 3-Testing class code

### 3.1 Testing on file1.txt

In [104]:
bigram_lm = LanguageModel(textfile="data/file1.txt")

Creating bigrams and unigrams list ...


0it [00:00, ?it/s]/home/tamer/anaconda3/envs/psut-nlp-course/lib/python3.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
5000it [01:36, 51.70it/s]


Laplacian smoothing ...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 52826/52826 [02:17<00:00, 383.43it/s]


In [105]:
test_string = "الاعدام لعامل مطعم قتل"
bigram_lm.run(test_string)

/home/tamer/anaconda3/envs/psut-nlp-course/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  after removing the cwd from sys.path.


['عشر', 'على', 'شخص', 'ملك', 'رب']

In [106]:
test_string = "عااااجل اعلنت روسيا"
bigram_lm.run(test_string)

/home/tamer/anaconda3/envs/psut-nlp-course/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  after removing the cwd from sys.path.


['يوم', 'صور', 'انضم', 'لصى', 'مصر']

In [107]:
test_string = "الملك عبدالله يعلن"
bigram_lm.run(test_string)

/home/tamer/anaconda3/envs/psut-nlp-course/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  after removing the cwd from sys.path.


['محام', 'عن', 'سلط', 'فرض', 'وفاة']

### 3.2 Testing on the combined textfile containing all the tweets: full_data.txt

Cancelled, since it was taking over 9 hours to complete creating the bigram model.